# 0. 降维概述
许多机器学习问题涉及成百上千的特征，应用庞大的特征量往往会造成许多问题：
1. 训练速度大大降低，很难寻找好的解决方案
2. 过高的数据维度更可能会产生过拟合，导致模型泛化能力弱。
3. 高维数据具有稀疏性，许多训练实例之间可能距离很远，找到数据规律更困难
解决以上问题，可以使用降维的方法。降维通过过滤掉一些不必要的冗余信息和细节信息，能够使模型更加准确，有效地加快训练速度。

# 1. 移除低方差特征
## 原理简析：
移除低方差是特征选择的一个基本方法。低方差说明该特征在每组数据间变化不大（极端情况下一直不变）。这样的特征对数据影响较小，可以优先剔除。
## 代码示例：

In [1]:
from sklearn.feature_selection import VarianceThreshold
# 示例：布尔值数据集，每个特征都是两点分布
X = [[0, 0, 1], [0, 1, 0], [1, 0, 0], [0, 1, 1], [0, 1, 0], [0, 1, 1]]
# 剔除某一个值概率超过0.8的特征 方差最小为0.8*0.2=0.16
sel = VarianceThreshold(threshold=0.16)
sel.fit_transform(X)

# 2. 单变量特征选择
单变量的特征选择是通过基于单变量的统计测试来选择最好的特征。简单的处理方法有：
1. 移除除了评分最高K个以外的特征`SelectKBest()`
2. p移除除了排名在一定百分比以外的特征`SelectPercentile()`
3. 对于单变量的检验方法，常用的有F检验，互信息方法（需要较多数据），卡方检验（只适用于分类问题）
   * 对于回归: f_regression , mutual_info_regression
   * 对于分类: chi2 , f_classif , mutual_info_classif

In [1]:
# 导入鸢尾花数据集
from sklearn.datasets import load_iris
iris = load_iris()
X, y = iris.data, iris.target
print(X.shape)
# 导入SelectKBest方法和检测指标f_classif（F检验）
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
X_new = SelectKBest(f_classif, k=2).fit_transform(X, y)
print(X_new.shape)

# 3. 递归式特征消除
递归式特征消除法使用某种估计器每次移除最不重要的特征，再在移除后的小集合中重复这个过程，通过交叉循环找到最佳的特征数量选择。
## 代码示例：递归式特征消除法进行重要性排名

In [2]:
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.feature_selection import RFE
import matplotlib.pyplot as plt
# 加载数据集
digits = load_digits()
X = digits.data
y = digits.target
print(X.shape)
# 创建RFE对象并进行排名
svc = SVC(kernel="linear", C=1)
rfe = RFE(estimator=svc, n_features_to_select=1, step=1)
rfe.fit(X, y)
ranking = rfe.ranking_.reshape(digits.images[0].shape)
# 根据排名ranking进行作图
plt.matshow(ranking)
plt.colorbar()
plt.title("Ranking of pixels with RFE")
plt.show()


## 代码示例：递归式特征消除法自动调整特征数

In [3]:
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.datasets import make_classification
# 使用三个特征进行交叉验证
X, y = make_classification(n_samples=1000, n_features=25, n_informative=3,
                           n_redundant=2, n_repeated=0, n_classes=8,
                           n_clusters_per_class=1, random_state=0)
# 创建RFE对象，计算交叉验证分数（准确度得分）
svc = SVC(kernel="linear")
# 使用rfecv函数，迭代到最佳特征数
min_features_to_select = 1 
rfecv = RFECV(estimator=svc, step=1, cv=StratifiedKFold(2),
              scoring='accuracy',
              min_features_to_select=min_features_to_select)
rfecv.fit(X, y)
print("Optimal number of features : %d" % rfecv.n_features_)

In [4]:
# 作图：每个特征的交叉验证分数
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(min_features_to_select,
               len(rfecv.grid_scores_) + min_features_to_select),
         rfecv.grid_scores_)
plt.show()

## 4. L1正则化方法
L1正则化方法能够将数据稀疏化，在数据更加分散的过程中，许多系数逐渐归0。该方法可以和SelectModel方法一起使用，选取在稀疏过程中没有为0系数的特征。

In [5]:
# 导入鸢尾花数据集
from sklearn.datasets import load_iris
iris = load_iris()
X, y = iris.data, iris.target
print(X.shape)
# 使用LinearSVC方法
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X, y)
model = SelectFromModel(lsvc, prefit=True)
X_new = model.transform(X)
print(X_new.shape)

## 5. 基于树的特征选取
使用树中的estimators模块，也能够消除一些不必要的特征

In [6]:
# 导入鸢尾花数据集
from sklearn.datasets import load_iris
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
iris = load_iris()
X, y = iris.data, iris.target
X.shape

In [7]:
clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(X, y)
clf.feature_importances_  

In [8]:
model = SelectFromModel(clf, prefit=True)
X_new = model.transform(X)
X_new.shape                         

# 参考资料：
sk-learn官方文档

# 作业
将卡方分布(chi2)为标准的单变量特征选择方法应用于乳腺癌数据集，输出包含前20个评分最高特征的数据集。

In [9]:
# 导入乳腺癌数据集
data = datasets.load_breast_cancer()
X = data.data
y = data.target